# Bertrand paradox
What is the probability that a *random* chord on the unit circle is longer than $\sqrt{3}$, the length of the side of an inscribed equilateral triangle?
The answer depends on the distribution of the random chord, i.e. on the sampling mechanism...

In [1]:
from IPython.core.display import HTML
display(HTML('<style>.container { width:95% !important; } </style>'))

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from tqdm.notebook import tqdm


SEED = 12345
rng = np.random.default_rng(seed=SEED)

In [3]:
theta0 = 0
theta1 = 2 * np.pi / 3
theta2 = 4 * np.pi / 3


def angle2coord(theta, r=1):
    return np.array([r * np.cos(theta), r * np.sin(theta)])


def unif_circle():
    Theta = rng.uniform(0, 2*np.pi, size=1)
    return angle2coord(Theta), Theta

# Warning: to generate the uniform distribution on the disk (2D),
# the radius should be sqrt of uniform.
def unif_disk():
    Theta = rng.uniform(0, 2*np.pi, size=1)
    R = np.sqrt(rng.uniform(0, 1, size=1))
    return angle2coord(Theta, R), R, Theta


A0 = angle2coord(theta0)
A1 = angle2coord(theta1)
A2 = angle2coord(theta2)

# Length of any side of the equilateral triangle A0A1A2 is sqrt(3)
assert np.isclose(np.sum((A2 - A1) ** 2), 3)

## Random endpoints

In [4]:
M = int(1e5)
res = np.empty(M)

for m in tqdm(range(M)):
    Theta1, Theta2 = rng.uniform(0, 2*np.pi, size=2)
    X1, _ = unif_circle() 
    X2, _ = unif_circle() 
    res[m] = np.sum((X2 - X1) ** 2) > 3
    
print("{:.1%} +/- {:.1%}".format(res.mean(), 1.96 * res.std() / np.sqrt(M)))

  0%|          | 0/100000 [00:00<?, ?it/s]

33.2% +/- 0.3%


## Random midpoint

In [5]:
M = int(1e5)
res = np.empty(M)

for m in tqdm(range(M)):
    X, R, Theta = unif_disk()
    
    # Endpoints of the unique chord for which X is the midpoint
    Theta1 = Theta + np.arccos(R)
    Theta2 = Theta - np.arccos(R)
    X1, X2 = angle2coord(Theta1), angle2coord(Theta2)
    res[m] = np.sum((X2 - X1) ** 2) > 3
    
print("{:.1%} +/- {:.1%}".format(res.mean(), 1.96 * res.std() / np.sqrt(M)))

  0%|          | 0/100000 [00:00<?, ?it/s]

24.9% +/- 0.3%
